In [ ]:
!pip install gradio pandas scikit-learn matplotlib seaborn folium shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
import folium
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
california_polygon = Polygon([
    (-124.48, 41.98), (-123.01, 41.98), (-122.84, 41.70), (-120.00, 41.70), (-117.03, 40.00),
    (-116.50, 36.00), (-114.13, 34.00), (-114.13, 32.53), (-117.00, 32.53), (-118.50, 34.00),
    (-120.50, 35.50), (-122.50, 37.50), (-124.48, 40.00), (-124.48, 41.98)
])

In [ ]:
df = pd.read_csv('/content/pems-8w.csv')

In [ ]:
if 'Timestamp' not in df.columns:
    df['Timestamp'] = pd.date_range(start='2023-01-01', periods=len(df), freq='H')
if 'Sensor_ID' not in df.columns:
    df['Sensor_ID'] = np.random.randint(1, 50, size=len(df))

df['Hour'] = pd.to_datetime(df['Timestamp']).dt.hour
df['Day'] = pd.to_datetime(df['Timestamp']).dt.dayofweek

if 'Congestion_Score' not in df.columns:
    df['Congestion_Score'] = np.random.randint(10, 100, size=len(df))

if 'Latitude' not in df.columns or 'Longitude' not in df.columns:
    df['Latitude'] = np.random.uniform(32.5, 42.0, size=len(df))
    df['Longitude'] = np.random.uniform(-124.5, -114.1, size=len(df))

df['is_land'] = df.apply(lambda row: california_polygon.contains(Point(row['Longitude'], row['Latitude'])), axis=1)
df = df[df['is_land']]

X = df[['Sensor_ID', 'Hour', 'Day']]
y = df['Congestion_Score']

<ipython-input-6-665314c8c2fe>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Timestamp'] = pd.date_range(start='2023-01-01', periods=len(df), freq='H')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
reg_model = RandomForestRegressor()
reg_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
def apply_clustering(df_local):
    cluster_data = df_local[['Latitude', 'Longitude', 'Congestion_Score']]
    kmeans = KMeans(n_clusters=3, random_state=42)
    df_local['Cluster'] = kmeans.fit_predict(cluster_data)
    return df_local, kmeans
df, kmeans = apply_clustering(df)

In [ ]:
def predict_congestion_and_update(sensor_id, hour, day):
    global df

    try:
        sensor_id = int(sensor_id)
        hour = int(hour)
        day = int(day)
    except:
        return "Invalid input. Please enter correct values.", None

    input_data = pd.DataFrame([[sensor_id, hour, day]], columns=['Sensor_ID', 'Hour', 'Day'])
    prediction = reg_model.predict(input_data)[0]
    congestion_level = "Low" if prediction < 40 else "Medium" if prediction < 70 else "High"

    df_updated = df.copy()
    df_updated['Hour'] = hour
    df_updated['Day'] = day
    X_dynamic = df_updated[['Sensor_ID', 'Hour', 'Day']]
    df_updated['Congestion_Score'] = reg_model.predict(X_dynamic)

    df_updated, new_kmeans = apply_clustering(df_updated)

    sensor_row = df_updated[df_updated['Sensor_ID'] == sensor_id]
    if not sensor_row.empty:
        cluster = sensor_row.iloc[0]['Cluster']
    else:
        cluster = new_kmeans.predict([[df['Latitude'].iloc[0], df['Longitude'].iloc[0], prediction]])[0]

    alt = df_updated[df_updated['Cluster'] != cluster].sample(3)
    alt_routes = "\n".join([f"Sensor: {r['Sensor_ID']} | Score: {r['Congestion_Score']:.2f}" for _, r in alt.iterrows()])

    df[:] = df_updated

    return f"Predicted Congestion Score: {prediction:.2f}\nLevel: {congestion_level}\n\n Alternate Route Suggestions:\n{alt_routes}", prediction

In [ ]:
def show_congestion_map(sensor_id=None, prediction_score=None):
    m = folium.Map(location=[36.0, -119.0], zoom_start=7)

    for _, row in df.iterrows():
        color = 'green' if row['Congestion_Score'] < 40 else 'orange' if row['Congestion_Score'] < 70 else 'red'

        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=5,
            popup=f"Sensor {row['Sensor_ID']} | Score: {row['Congestion_Score']:.2f}",
            color=color,
            fill=True,
            fill_color=color
        ).add_to(m)

    if sensor_id is not None and prediction_score is not None:
        selected = df[df['Sensor_ID'] == int(sensor_id)]
        if not selected.empty:
            lat = selected.iloc[0]['Latitude']
            lon = selected.iloc[0]['Longitude']
            pred_color = 'green' if prediction_score < 40 else 'orange' if prediction_score < 70 else 'red'
            folium.Marker(
                location=[lat, lon],
                popup=f" Selected Sensor {sensor_id} | Predicted Score: {prediction_score:.2f}",
                icon=folium.Icon(color=pred_color, icon='info-sign')
            ).add_to(m)

    m.save('/content/congestion_map_california_dynamic.html')
    return '/content/congestion_map_california_dynamic.html'

sensor_options = []
unique_sensors = df[['Sensor_ID', 'Latitude', 'Longitude']].drop_duplicates()
for _, row in unique_sensors.iterrows():
    sensor_options.append(f"Sensor {int(row['Sensor_ID'])} (Lat: {row['Latitude']:.2f}, Lon: {row['Longitude']:.2f})")


In [ ]:
def full_interface(sensor_dropdown, hour, day):
    sensor_id = int(sensor_dropdown.split()[1])
    result_text, pred_score = predict_congestion_and_update(sensor_id, hour, day)
    map_file = show_congestion_map(sensor_id, pred_score)
    return result_text, map_file

gr.Interface(
    fn=full_interface,
    inputs=[
        gr.Dropdown(choices=sensor_options, label="Select Sensor (ID + Coordinates)"),
        gr.Slider(0, 23, label="Hour of Day"),
        gr.Radio(choices=[0, 1, 2, 3, 4, 5, 6], label="Day of Week (0=Mon, 6=Sun)")
    ],
    outputs=[
        gr.Textbox(label="Prediction & Alternate Routes"),
        gr.File(label="Traffic Map")
    ],
    title="Traffic Monitoring and Management System ",
    description="Predicts congestion dynamically for all sensors."
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e3f09c56ddb4e2d3d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
